In [1]:
import json
import pandas as pd
database_route = "../Database/"
filemap = {
    "NYSE"  : database_route + "NYSE.csv",
    "NASDAQ": database_route + 'NASDAQ.csv',
    "SP500" : database_route + "S&P500_stocks.csv",
    "Yuanta": database_route + "Yuanta_us_stocks.csv"
}
def get_tickers(file="SP500"):
    
    # f = open(filemap[file])
    # dict = pd.Series(json.load(f))
    dict = pd.read_csv(filemap[file], header=None)
    if file in ["NYSE", "NASDAQ"]:
        dict = dict.iloc[1:, 0]
    if file in ["SP500", "Yuanta"]:
        dict = dict.iloc[:, 0]
    return dict

a = get_tickers("Yuanta")

WPR_2MA vs WPR_2MA_more_condition:
higher return/lower win rates vs. lower return/higher win rate

In [37]:
from backtesting import Backtest, Strategy
import yfinance as yf
import json
import pandas as pd
filemap = {
    "NYSE"  : "nyse_tickers.json",
    "NASDAQ": 'nasdaq_tickers.json',
    "SP500" : "S&P500_stocks.csv",
    "Yuanta": "Yuanta_us_stocks.csv"
}
def get_tickers(file="SP500"):
    if file in ["NYSE", "NASDAQ"]:
        f = open(filemap[file])
        dict = pd.Series(json.load(f))
    if file in ["SP500", "Yuanta"]:
        dict = pd.read_csv(filemap[file], header=None)
        dict = dict.iloc[:, 0]
    return dict


def WPR(data):
    period = 12
    max = data.Close.s.rolling(period).max()
    min = data.Close.s.rolling(period).min()
    wpr = (max - data.Close.s)/(max - min) * (-100) 
    return wpr

def moving_average_exp(data, period):
    return data.Close.s.ewm(period).mean()

def close_price(data):
    return data.Close.s

# strict WPR_2MA usage
class WPR_2MA_more_condition(Strategy):
    # parameters
    period_short_ma = 14
    period_long_ma = 28
    tp_factor = 1.2
    sl_factor = 0.9
    def init(self):
        self.short_ma = self.I(moving_average_exp, self.data, period=period_short_ma)
        self.long_ma = self.I(moving_average_exp, self.data, period=period_long_ma)
        self.wpr = self.I(WPR, self.data)
        self.close = self.I(close_price, self.data)
    def next(self):
        # parameters
        tp_factor = self.tp_factor
        sl_factor = self.sl_factor
        # ma 
        long_ma0 = self.long_ma[-1]
        long_ma1 = self.long_ma[-2]
        short_ma0 = self.short_ma[-1]
        short_ma1 = self.short_ma[-2]
        death_crossover = (long_ma0 > short_ma0) and (long_ma1 < short_ma1)
        golden_crossover = (long_ma0 < short_ma0) and (long_ma1 > short_ma1)
        long_signal = (long_ma0 < short_ma0) and (long_ma1 < short_ma1)
        
        # wpr
        wpr0 = self.wpr[-1]
        wpr1 = self.wpr[-2]
        wpr2 = self.wpr[-3]
        wpr_lower_cross = wpr0 > -80 and wpr1 < -80
        wpr_lower_sustain = wpr2 < -80 and wpr1 < -80
        # price
        close0 = self.close[-1]
        close1 = self.close[-2]
        price_over_ma = close0 > long_ma0 and close1 > long_ma1
        close_price = self.close[-1]
        if self.position:
            if death_crossover:
                self.position.close()

        else:
            if long_signal and wpr_lower_cross and wpr_lower_sustain and price_over_ma:
                self.buy(tp=close_price*tp_factor, sl=close_price*sl_factor)

# default WPR_2MA usage
class WPR_2MA(Strategy):
    # parameters
    period_short_ma = 16
    period_long_ma = 20
    tp_factor = 1.2
    sl_factor = 0.9
    def init(self):
        period_long_ma = self.period_long_ma
        period_short_ma = self.period_short_ma
        self.short_ma = self.I(moving_average_exp, self.data, period=period_short_ma)
        self.long_ma = self.I(moving_average_exp, self.data, period=period_long_ma)
        self.wpr = self.I(WPR, self.data)
        self.close = self.I(close_price, self.data)
    def next(self):
        # paramaters
        tp_factor = self.tp_factor
        sl_factor = self.sl_factor
        # ma 
        long_ma0 = self.long_ma[-1]
        long_ma1 = self.long_ma[-2]
        short_ma0 = self.short_ma[-1]
        short_ma1 = self.short_ma[-2]
        death_crossover = (long_ma0 > short_ma0) and (long_ma1 < short_ma1)
        golden_crossover = (long_ma0 < short_ma0) and (long_ma1 > short_ma1)
        long_signal = (long_ma0 < short_ma0) and (long_ma1 < short_ma1)
        
        # wpr
        wpr0 = self.wpr[-1]
        wpr1 = self.wpr[-2]
        wpr_lower_cross = wpr0 > -80 and wpr1 < -80
        wpr_upper_cross = wpr0 < -20 and wpr1 > -20
        close_price = self.close[-1]
        if self.position:
            if death_crossover:
                self.position.close()

        else:
            if long_signal and wpr_lower_cross:
                self.buy(tp=close_price*tp_factor, sl=close_price*sl_factor)


# =======================================
result = {}
backtest_list = get_tickers("SP500")[:]
# backtest_list = ["GPS", "LYV", "LKQ", "IP", "ZBRA", "PNR", "MGM", "CRL", "MTCH", "HII", "BG", "IPG", "MRO"]
# backtest_list = ["GPS"]
for ticker in backtest_list:
    df = yf.download(ticker.split(), start="2020-01-01", interval="1d", progress=False)
    # print(df.head())
    # print("=========")
    # df = pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv", index_col="Date")
    # print(df.head())
    try:
        bt = Backtest(df, WPR_2MA, commission=0.002, exclusive_orders=True)
        # stats, heatmap = bt.optimize(period_short_ma=[i for i in range(7, 22)],
        #         period_long_ma=[i for i in range(14, 29)], 
        #         maximize='Return [%]', max_tries=300,
        #         random_state=0,
        #         return_heatmap=True)
        # bt.plot()
        stats = bt.run()
        elements = [6, 7, 8, 10, 13, 17, 18, 23]
        result[ticker] = {stats.keys()[i] : stats.values[i] for i in elements}
    except ValueError:
        print(ticker + " doesn't exist, passed")
        pass

In [44]:
# stats
# bt.plot()
a = pd.DataFrame(result)
result_con = {
    "average_win_rate" : a.T.iloc[:, -2].mean(),
    "average_return" : a.T.iloc[:, 0].mean(),
    "annual_return": a.T.iloc[:, 2].mean(),
    "Average Sharpe": a.T.iloc[:, 3][a.T.iloc[:, 3]!= 0].mean()
}
print(result_con)
a.T.iloc[:, 0].max()
for key, value in result_con:
    print(key, ":", value)


{'average_win_rate': 36.21297061978825, 'average_return': 10.25005712904708, 'annual_return': 1.3750204870981158, 'Average Sharpe': 0.3558952951292188}


ValueError: too many values to unpack (expected 2)

In [42]:
print("better: ", len(a.T[a.T.iloc[:, 0] > a.T.iloc[:, 1]]))
print("worse: ", len(a.T[a.T.iloc[:, 0] < a.T.iloc[:, 1]]))


better:  146
worse:  357


In [45]:
a.T[a.T.iloc[:, 0] < a.T.iloc[:, 1]]

,Return [%],Buy & Hold Return [%],Return (Ann.) [%],Sharpe Ratio,Max. Drawdown [%],# Trades,Win Rate [%],Avg. Trade Duration
MSFT,69.184602,161.829162,13.672306,0.642395,-24.849757,8,62.5,92 days
AAPL,-12.12591,151.506579,-3.101263,0.0,-35.935229,15,33.333333,35 days
NVDA,45.348788,1102.667677,9.542309,0.319017,-48.004979,21,42.857143,28 days
AMZN,-3.690103,83.824112,-0.912156,0.0,-45.19357,15,26.666667,34 days
GOOGL,77.694737,117.728036,15.040066,0.723343,-21.472476,9,66.666667,78 days
...,...,...,...,...,...,...,...,...
PNW,-53.321063,-24.473001,-16.946061,0.0,-56.248847,11,0.0,45 days
BEN,-17.377992,5.818039,-4.545788,0.0,-42.003224,13,23.076923,33 days
DVA,11.086112,45.993356,2.595405,0.127524,-24.561633,9,44.444444,71 days
RL,16.438348,47.32721,3.778768,0.218608,-27.762766,7,42.857143,43 days


In [29]:
bt.plot()

Row(id='4002', ...)

In [17]:
a = pd.DataFrame(result)
a.T.iloc[:50, :]

,Return [%],Buy & Hold Return [%],Return (Ann.) [%],Sharpe Ratio,Max. Drawdown [%],# Trades,Win Rate [%],Avg. Trade Duration
POOL,-17.140742,80.838127,-4.479059,0.0,-47.385358,11,36.363636,43 days
DGX,-18.330687,19.871365,-4.815212,0.0,-24.927771,9,11.111111,64 days
ENPH,-32.586593,317.416498,-9.16293,0.0,-60.907891,22,31.818182,14 days
VTRS,-35.703136,-43.438254,-10.20478,0.0,-35.703136,6,0.0,41 days
BBY,-34.645028,-13.41198,-9.846858,0.0,-48.594426,8,25.0,48 days
CAG,-31.777334,-18.52513,-8.898372,0.0,-40.474356,10,20.0,53 days
AMCR,-12.820758,-14.393224,-3.28856,0.0,-30.167831,9,11.111111,55 days
KEY,-0.943935,-31.441263,-0.230876,0.0,-42.934859,12,33.333333,35 days
NDSN,-9.562257,59.553357,-2.419772,0.0,-33.952149,8,25.0,82 days
UAL,-56.479661,-52.830396,-18.352227,0.0,-58.868163,13,7.692308,25 days
